## Import packages and Define source and result file locations

In [1]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle

sys.path.insert(1, '../pkgs')
import ml_functions as mlfuncs
import ml_clean_feature as mlclean
import ml_clean_config as mlconfigs

In [2]:
# Path to results
year = 2021
source_path     = "../data/brfss/"
source_file     = mlfuncs.brfss_parquet_file( source_path, year )

result_path     = "../data/"
report_file     = 'feature_report.pkl'

## Analyze the set of all candidate features

- Reduce full set of features to just the candidate features
- Check for features that have too many null values (>70K)
- Remove features that have too many null values
- Move on to cleaning of each feature

In [3]:
# Full candidate  dataset
diabetes_features_2021_all_candidates = [
    'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'PRIMINSR', 'PERSDOC3', 
    'CHECKUP1', 'EXERANY2', 'BPHIGH6', 'BPMEDS', 'CHOLCHK3', 
    'TOLDHI3', 'CHOLMED3', 'CVDCRHD4', 'CVDSTRK3', 'ADDEPEV3', 
    'CHCKDNY2', 'DIABETE4', 'MARITAL', 'EDUCA', 'RENTHOM1', 
    'EMPLOY1', 'INCOME3', 'WEIGHT2', 'DEAF', 'BLIND', 
    'DIFFWALK', 'FLUSHOT7', 'PREDIAB1', 'CHKHEMO3', 'EYEEXAM1', 
    'TOLDCFS', 'HAVECFS', 'TOLDHEPC', 'HAVEHEPB', 'HPVADVC4', 
    'SHINGLE2', 'CIMEMLOS', 'CDDISCUS', 'MSCODE', '_IMPRACE', 
    '_RFHLTH', '_HLTHPLN', '_TOTINDA', '_MICHD', '_PRACE1', 
    '_RACE', '_RACEGR3', '_SEX', '_AGEG5YR', 'WTKG3', 
    '_BMI5', '_BMI5CAT', '_EDUCAG', '_INCOMG1', '_SMOKER3', 
    '_RFSMOK3', '_CURECI1', '_DRNKWK1', '_RFDRHV7', 'FTJUDA2_', 
    'FRUTDA2_', 'GRENDA1_', 'FRNCHDA_', 'POTADA1_', 'VEGEDA2_', 
    '_FRUTSU1', '_VEGESU1', '_FRTLT1A']


In [4]:
# Read in full BRFSS 2021 dataset
df = pd.read_parquet(source_file, engine="fastparquet")


In [5]:
# Create feature dataframe with only candidate diabetes features
feature_df = df[diabetes_features_2021_all_candidates].copy()

In [6]:
# Describe the data
feature_df.describe()


,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINSR,PERSDOC3,CHECKUP1,EXERANY2,BPHIGH6,BPMEDS,CHOLCHK3,...,_RFDRHV7,FTJUDA2_,FRUTDA2_,GRENDA1_,FRNCHDA_,POTADA1_,VEGEDA2_,_FRUTSU1,_VEGESU1,_FRTLT1A
count,438689.000000,438690.000000,438691.000000,438690.000000,438691.00000,438691.000000,438691.000000,438691.000000,172133.000000,438691.000000,...,438693.000000,3.943440e+05,3.947420e+05,3.944430e+05,3.939280e+05,3.902530e+05,3.901650e+05,3.876060e+05,3.785660e+05,438693.000000
mean,2.524761,63.190139,59.923347,10.614445,1.57887,1.470787,1.258043,2.238437,1.187861,2.720555,...,1.692548,4.459572e+01,1.341733e+02,7.890303e+01,2.571366e+01,2.851175e+01,1.396707e+02,1.783430e+02,2.715442e+02,2.270561
std,1.082066,36.222075,37.472680,24.795124,0.89225,1.128487,0.522586,1.053716,0.507136,1.717564,...,2.163298,3.652686e+02,5.006682e+02,4.634876e+02,1.736687e+02,1.787448e+02,6.511466e+02,6.912931e+02,1.036227e+03,2.485479
min,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.000000
25%,2.000000,25.000000,15.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,5.397605e-79,4.300000e+01,1.400000e+01,3.000000e+00,7.000000e+00,4.300000e+01,5.700000e+01,1.140000e+02,1.000000
50%,2.000000,88.000000,88.000000,3.000000,1.00000,1.000000,1.000000,3.000000,1.000000,2.000000,...,1.000000,3.000000e+00,1.000000e+02,4.300000e+01,1.400000e+01,1.400000e+01,1.000000e+02,1.000000e+02,1.670000e+02,1.000000
75%,3.000000,88.000000,88.000000,3.000000,2.00000,1.000000,1.000000,3.000000,1.000000,3.000000,...,1.000000,3.300000e+01,1.000000e+02,7.100000e+01,2.900000e+01,2.900000e+01,1.000000e+02,2.000000e+02,2.290000e+02,2.000000
max,9.000000,99.000000,99.000000,99.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.900000e+03,9.900000e+03,9.900000e+03,9.900000e+03,9.900000e+03,9.900000e+03,1.980000e+04,3.960000e+04,9.000000


In [7]:
# Determine features with too many null values (>70K)
cnt = 0
max_allowed_nulls = 70000
maxed_nulls = []
for column in feature_df.columns:
    cnt += 1
    tot = feature_df[column].isna().sum()
    print(f"Feature[{cnt}]: {column}: {tot}")
    if tot > max_allowed_nulls:
        maxed_nulls.append(f'{column}')

print (f"features that exceeded max nulls: \n{maxed_nulls}")

Feature[1]: GENHLTH: 4
Feature[2]: PHYSHLTH: 3
Feature[3]: MENTHLTH: 2
Feature[4]: PRIMINSR: 3
Feature[5]: PERSDOC3: 2
Feature[6]: CHECKUP1: 2
Feature[7]: EXERANY2: 2
Feature[8]: BPHIGH6: 2
Feature[9]: BPMEDS: 266560
Feature[10]: CHOLCHK3: 2
Feature[11]: TOLDHI3: 60836
Feature[12]: CHOLMED3: 61571
Feature[13]: CVDCRHD4: 2
Feature[14]: CVDSTRK3: 2
Feature[15]: ADDEPEV3: 3
Feature[16]: CHCKDNY2: 3
Feature[17]: DIABETE4: 3
Feature[18]: MARITAL: 5
Feature[19]: EDUCA: 5
Feature[20]: RENTHOM1: 8
Feature[21]: EMPLOY1: 3588
Feature[22]: INCOME3: 8847
Feature[23]: WEIGHT2: 11816
Feature[24]: DEAF: 14602
Feature[25]: BLIND: 15744
Feature[26]: DIFFWALK: 18009
Feature[27]: FLUSHOT7: 27648
Feature[28]: PREDIAB1: 324076
Feature[29]: CHKHEMO3: 416464
Feature[30]: EYEEXAM1: 416468
Feature[31]: TOLDCFS: 438693
Feature[32]: HAVECFS: 438693
Feature[33]: TOLDHEPC: 427889
Feature[34]: HAVEHEPB: 427914
Feature[35]: HPVADVC4: 425379
Feature[36]: SHINGLE2: 425141
Feature[37]: CIMEMLOS: 405542
Feature[38]: CDD

In [8]:
# Create final set of 2021 features for use in creating the diabetes_features.md in step 3.
diabetes_features_2021 = []
for feature in diabetes_features_2021_all_candidates:
    if feature not in maxed_nulls:
        diabetes_features_2021.append(feature)

print(diabetes_features_2021)

['GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'PRIMINSR', 'PERSDOC3', 'CHECKUP1', 'EXERANY2', 'BPHIGH6', 'CHOLCHK3', 'TOLDHI3', 'CHOLMED3', 'CVDCRHD4', 'CVDSTRK3', 'ADDEPEV3', 'CHCKDNY2', 'DIABETE4', 'MARITAL', 'EDUCA', 'RENTHOM1', 'EMPLOY1', 'INCOME3', 'WEIGHT2', 'DEAF', 'BLIND', 'DIFFWALK', 'FLUSHOT7', '_IMPRACE', '_RFHLTH', '_HLTHPLN', '_TOTINDA', '_MICHD', '_PRACE1', '_RACE', '_RACEGR3', '_SEX', '_AGEG5YR', 'WTKG3', '_BMI5', '_BMI5CAT', '_EDUCAG', '_INCOMG1', '_SMOKER3', '_RFSMOK3', '_CURECI1', '_DRNKWK1', '_RFDRHV7', 'FTJUDA2_', 'FRUTDA2_', 'GRENDA1_', 'FRNCHDA_', 'POTADA1_', 'VEGEDA2_', '_FRUTSU1', '_VEGESU1', '_FRTLT1A']


In [9]:
# Drop featurs with too many null values
feature_df.drop(maxed_nulls, axis=1, inplace=True)
feature_df

,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINSR,PERSDOC3,CHECKUP1,EXERANY2,BPHIGH6,CHOLCHK3,TOLDHI3,...,_RFDRHV7,FTJUDA2_,FRUTDA2_,GRENDA1_,FRNCHDA_,POTADA1_,VEGEDA2_,_FRUTSU1,_VEGESU1,_FRTLT1A
0,5.0,20.0,10.0,3.0,1.0,2.0,2.0,3.0,2.0,1.0,...,1.0,5.397605e-79,100.0,5.700000e+01,4.300000e+01,14.0,100.0,100.0,214.0,1.0
1,3.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,...,1.0,5.397605e-79,100.0,1.400000e+01,5.397605e-79,14.0,100.0,100.0,128.0,1.0
2,2.0,88.0,88.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,...,1.0,5.397605e-79,100.0,5.397605e-79,1.400000e+01,14.0,43.0,100.0,71.0,1.0
3,2.0,88.0,10.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,7.100000e+01,43.0,1.000000e+01,5.700000e+01,27.0,71.0,114.0,165.0,1.0
4,5.0,30.0,88.0,3.0,1.0,1.0,1.0,4.0,2.0,1.0,...,1.0,5.397605e-79,100.0,1.000000e+02,2.900000e+01,29.0,100.0,100.0,258.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438688,2.0,88.0,20.0,88.0,3.0,4.0,1.0,1.0,6.0,2.0,...,1.0,1.400000e+01,143.0,2.900000e+01,1.400000e+01,50.0,300.0,157.0,393.0,1.0
438689,3.0,88.0,88.0,77.0,1.0,1.0,2.0,1.0,2.0,2.0,...,1.0,1.000000e+02,100.0,1.000000e+02,1.400000e+01,14.0,29.0,200.0,157.0,1.0
438690,2.0,88.0,88.0,10.0,1.0,1.0,1.0,3.0,2.0,1.0,...,1.0,5.397605e-79,200.0,2.900000e+01,5.397605e-79,14.0,100.0,200.0,143.0,1.0
438691,2.0,88.0,88.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,...,1.0,5.397605e-79,100.0,4.300000e+01,5.397605e-79,13.0,100.0,100.0,156.0,1.0


In [10]:
# Drop all rows with null values
feature_df.dropna(inplace=True)

In [11]:
feature_df

,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINSR,PERSDOC3,CHECKUP1,EXERANY2,BPHIGH6,CHOLCHK3,TOLDHI3,...,_RFDRHV7,FTJUDA2_,FRUTDA2_,GRENDA1_,FRNCHDA_,POTADA1_,VEGEDA2_,_FRUTSU1,_VEGESU1,_FRTLT1A
0,5.0,20.0,10.0,3.0,1.0,2.0,2.0,3.0,2.0,1.0,...,1.0,5.397605e-79,100.0,5.700000e+01,4.300000e+01,14.0,100.0,100.0,214.0,1.0
2,2.0,88.0,88.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,...,1.0,5.397605e-79,100.0,5.397605e-79,1.400000e+01,14.0,43.0,100.0,71.0,1.0
3,2.0,88.0,10.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,7.100000e+01,43.0,1.000000e+01,5.700000e+01,27.0,71.0,114.0,165.0,1.0
4,5.0,30.0,88.0,3.0,1.0,1.0,1.0,4.0,2.0,1.0,...,1.0,5.397605e-79,100.0,1.000000e+02,2.900000e+01,29.0,100.0,100.0,258.0,1.0
5,3.0,88.0,88.0,3.0,1.0,1.0,2.0,3.0,2.0,2.0,...,1.0,5.397605e-79,29.0,1.400000e+01,5.397605e-79,14.0,14.0,29.0,42.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438687,4.0,88.0,88.0,3.0,1.0,3.0,1.0,1.0,4.0,1.0,...,1.0,2.000000e+02,50.0,2.000000e+02,1.400000e+01,14.0,71.0,250.0,299.0,1.0
438688,2.0,88.0,20.0,88.0,3.0,4.0,1.0,1.0,6.0,2.0,...,1.0,1.400000e+01,143.0,2.900000e+01,1.400000e+01,50.0,300.0,157.0,393.0,1.0
438690,2.0,88.0,88.0,10.0,1.0,1.0,1.0,3.0,2.0,1.0,...,1.0,5.397605e-79,200.0,2.900000e+01,5.397605e-79,14.0,100.0,200.0,143.0,1.0
438691,2.0,88.0,88.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,...,1.0,5.397605e-79,100.0,4.300000e+01,5.397605e-79,13.0,100.0,100.0,156.0,1.0


#### Initial cleaning results
Resulting dataframe has 297898 Rows with 54 Features and 1 target (DIABETE4)

---

### Evaluate and clean each column:

- refer to the feature report ([diabetes_features.md](../diabetes_features.md)) generated in step 3 of data cleaning:


In [12]:
# GENHLTH: General Health
# Keep values 1..5 (Excellent to Poor)
# Remove values 7, 8, BLANK  (Don't know, Refused, blank)
#feature_df = ['GENHLTH'] = feature_df = ['GENHLTH'].replace({2:0, 3:0, 1:2, 4:1})


In [13]:
clean_configs = mlconfigs.clean_configurations()
#clean_configs

In [17]:
for column in feature_df:
    if column in clean_configs:
        print(f"Cleaning Feature: {column}")
        clean_config = clean_configs[column]
        
        if 'values_to_drop' in clean_config:
            params = clean_config['values_to_drop']
            if not params:
                print(f"  {'values_to_drop'}: ********* NO Parameters were specified *********")
            else:
                print(f"  {'values_to_drop'}: {params}")
        
        if 'translate' in clean_config:
            params = clean_config['translate']
            if not params:
                print(f"  {'translate'}: ********* NO Parameters were specified *********")
            else:
                print(f"  {'translate'}: {params}")
        
        if 'scale' in clean_config:
            params = clean_config['scale']
            if not params:
                print(f"  {'scale'}: ********* NO Parameters were specified *********")
            else:
                print(f"  {'scale'}: {params}")

        # for clean in clean_config:
        #     params = clean_config[clean]
        #     if (clean == 'scale'):
        #         print(f"  {clean}: {params}")
        #     if (clean == 'translate'):
        #         print(f"  {clean}: {params}")
        #     if (clean == 'values_to_drop'):
        #         print(f"  {clean}: {params}")
#        if ('')
    else:
        print(f"Feature DOES NOT exist: {column}")

    print(f"Unique features in [{column}]:  {feature_df[column].unique()}")


Cleaning Feature: GENHLTH
  values_to_drop: [4]
  translate: {1: 0, 2: 1}
  scale: {'div': 100, 'round': 2}
Unique features in [GENHLTH]:  [5. 2. 3. 4. 1. 7. 9.]
Cleaning Feature: PHYSHLTH
  values_to_drop: ********* NO Parameters were specified *********
  translate: ********* NO Parameters were specified *********
  scale: ********* NO Parameters were specified *********
Unique features in [PHYSHLTH]:  [20. 88. 30. 25.  7. 77.  1. 10.  4.  2.  3. 15.  8. 13. 14.  5.  6. 12.
 21. 24. 29. 18. 99.  9. 16. 17. 26. 28. 27. 11. 19. 22. 23.]
Cleaning Feature: MENTHLTH
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: {'div': 100, 'round': 2}
Unique features in [MENTHLTH]:  [10. 88.  5. 25. 15. 77.  2.  7. 30.  3. 14. 20.  8.  1.  4. 99. 21. 28.
 24. 12.  6. 22. 27. 18. 13. 17. 16.  9. 19. 29. 23. 11. 26.]
Cleaning Feature: PRIMINSR
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: {'div':

### Rename Columns:



### Clean Types